In [1]:
import sys
import random
from configparser import ConfigParser

config = ConfigParser()
config.read("config.ini")
CARLA_PATH = config.get("main", "CARLA_PATH")
# Enable import of 'carla'
sys.path.append(CARLA_PATH + "PythonAPI/carla/dist/carla-0.9.9-py3.6-linux-x86_64.egg")
# Enable import of 'agents' and it's submodules
sys.path.insert(0, CARLA_PATH + "PythonAPI/carla/")
# Enable import of utilities from GlobalPathPlanner
sys.path.insert(0, "../../../global_route_planner/")

import carla

from agents.navigation.local_planner import LocalPlanner, _compute_connection

from global_planner import get_client, spawn_vehicle, draw_waypoints, get_global_planner

import numpy as np

### Initialize client, CARLA world, and a global route planner

In [2]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

# Get global planner
grp = get_global_planner(world, 1)  ## Discretization of 1

Connection to CARLA server established!


In [3]:
spawn_point = random.choice(world.get_map().get_spawn_points())
start_waypoint = world.get_map().get_waypoint(spawn_point.location,project_to_road=True)
start_waypoint.transform.location.z += 0.1
draw_waypoints(world, [start_waypoint], 20)

In [4]:
start_waypoint.transform.rotation.yaw

89.78770446777344

### Create a start point

In [5]:
start_x = 54.04038619995117
start_y = -192.64503479003906
start_z = 3.530714869499207
start_yaw = 1.4395599365234375

start_location = carla.Location(x=start_x, y=start_y, z=start_z)
start_rotation = carla.Rotation(yaw=start_yaw, roll=0, pitch=0)
start_point = carla.Transform(location=start_location, rotation=start_rotation)
start_waypoint = world.get_map().get_waypoint(start_point.location,project_to_road=True)
draw_waypoints(world, [start_waypoint], 20)

In [6]:
# vehicle_blueprint = world.get_blueprint_library().filter("model3")[0]
actor = spawn_vehicle(world, "model3")

In [ ]:
def get_turn_direction(vector_current, vector_next):
    
    vector_current = np.array([vector_current.x, vector_current.y])
    vector_next = np.array([vector_next.x, vector_next.y])

    cos_theta = np.dot(vector_current, vector_next) / (np.linalg.norm(vector_current)*np.linalg.norm(vector_next))
    angle_rad = np.arccos(cos_theta)
    
    return np.rad2deg(angle_rad)

In [12]:
traffic_light_list = world.get_actors().filter('traffic.traffic_light*')

In [13]:
tl1 = traffic_light_list[0]

In [29]:
tp = world.get_map().get_waypoint(tl1.get_location(),project_to_road=True)
draw_waypoints(world, [tp], 10)

In [31]:
tl1.trigger_volume

In [30]:
tmp_transform = tl1.get_transform()
tmp_bounding_box = tl1.trigger_volume
tmp_bounding_box.location += tmp_transform.location
world.debug.draw_box(tmp_bounding_box, tmp_transform.rotation, life_time=5, color=carla.Color(r=128, g=0, b=128))

### Spawn an actor at the start point

In [7]:
# actor = spawn_vehicle(world, "model3", start_waypoint.transform)
current_waypoint = world.get_map().get_waypoint(actor.get_location(),project_to_road=True)

In [8]:
draw_waypoints(world, [current_waypoint])

In [ ]:
points_on_actor_lane = current_waypoint.next_until_lane_end(1)
last_point_on_lane = points_on_actor_lane[-1]

connections_to_last_point_on_lane = last_point_on_lane.next(1)

connecting_roads = []
for point in connections_to_last_point_on_lane:
    connecting_roads.append(point.next_until_lane_end(1))
    

In [ ]:
draw_waypoints(world, [current_waypoint])

In [ ]:
this_jn = connecting_roads[0][0].get_junction()
pairs_of_waypoints_on_jn = this_jn.get_waypoints(carla.LaneType.Driving)

In [ ]:
[(p1.road_id, p2.road_id) for (p1,p2) in pairs_of_waypoints_on_jn]

In [ ]:
points_on_actor_lane[0].road_id

In [ ]:
idx = 1
draw_waypoints(world, pairs_of_waypoints_on_jn[idx], 10)
get_turn_direction(pairs_of_waypoints_on_jn[idx][0].transform.rotation.get_forward_vector(), pairs_of_waypoints_on_jn[idx][1].transform.rotation.get_forward_vector())

In [ ]:
waypoints = [[start_waypoint, None, None]]

queue = []
queue.append(start_waypoint)
while(len(waypoints) != 1000 and len(queue) != 0):
    current_waypoint = queue.pop(0)
    next_waypoints = list(current_waypoint.next(1))
    queue += next_waypoints
    for next_waypoint in next_waypoints:
        waypoints.append([current_waypoint, next_waypoint, _compute_connection(current_waypoint, next_waypoint)])
        


In [ ]:
draw_waypoints(world, [waypoint[0] for waypoint in waypoints], life_time=50)

In [ ]:
# draw_waypoints(world, [waypoints[0][0], waypoints[34][0]], life_time=5)

In [ ]:
start_location = waypoints[0][0].transform.location
end_location = waypoints[34][0].transform.location

In [ ]:
route = grp.trace_route(start_location, end_location)
route = [(item[0], item[1], item[0].road_id) for item in route]

In [ ]:
draw_waypoints(world, [item[0] for item in route], 15)

In [ ]:
route

In [ ]:
d = {}
for waypoint_tuple in waypoints[:100]:
    
    tmp = waypoint_tuple[0]
    if(tmp.road_id in d.keys()):
        d[tmp.road_id].append(tmp)
    else:
        d[tmp.road_id] = [tmp]
    

In [ ]:
d

In [ ]:
gp = grp._graph
gp.edges[257,394]

In [ ]:
def draw_grp_graph_edge(edge):
    
    draw_waypoints(world, [edge['entry_waypoint'], edge['exit_waypoint']])

In [ ]:
draw_grp_graph_edge(gp.edges[257,394])

In [ ]:
        next_edge = self._graph.edges[current_node, next_node]


In [ ]:
waypoints_on_junction = []
for waypoint in waypoints:
    if waypoint[0].get_junction() is not None:
        waypoints_on_junction.append(waypoint[0])

In [ ]:
draw_waypoints(world, waypoints_on_junction)

In [ ]:
def view_node(graph, idx):
    

In [ ]:
ll.